In [27]:
from importations import *
from classes import * 

from iteration_valeurs import * 
from smc import *
from q_learning import *
from visualisation import visu_random_walk
from accessibilite import *

#test
from random_walk import walk


In [28]:

file = "../exemple/ex.mdp"

lexer = gramLexer(FileStream(file))
stream = CommonTokenStream(lexer)
parser = gramParser(stream)
tree = parser.program()
data_listener = gramDataListener()
walker = ParseTreeWalker()
walker.walk(data_listener, tree)

#création de states, actions, transitions
states = data_listener.states
actions = data_listener.actions
transitions = data_listener.transitions
statesid = [state.id for state in states]
statesgain = [state.gain for state in states]

print(f"Liste des états : {statesid}")
print(f"Liste des gains : {statesgain}")
print()
print(actions)
print()
for transition in transitions:
    print(transition)
print()

#Vérification : chaque transition est effectuée avec une action définie
for transition in transitions:
    if str(transition.action) not in str(actions) and transition.action.id != '':
        raise ValueError(f"{transition.action.id} n\'est pas une action valide")
    
#Vérification : chaque transition part d'un état bien défini vers un état bien défini
for transition in transitions:
    if str(transition.departure.id) not in str(statesid) :
                raise ValueError(f"{transition.departure.id} n\'est pas un état valide")
            
    for target in transition.targets : 
        if str(target.id) not in str(statesid) : 
                raise ValueError(f"{target.id} n\'est pas un état valide")
            
#Vérif pas déterministe et non-déterministe
l1 = []
l2 = []
for transition in transitions:
    if  transition.action.id != '':
        l1.append(transition.departure.id)
    else :
        l2.append(transition.departure.id)
intersection = [x for x in l2 if x in l1]
if len (intersection) !=0 : 
        raise ValueError(f"{intersection[0]} est à la fois déterministe et non-déterministe.")
        
#Vérif : chaque état à une tansition
e = []
for transition in transitions:
    if transition.departure.id not in e : 
        e.append(transition.departure.id)
if len(e) != len(states) : 
    l = [x.id for x in states if x.id not in e]
    raise ValueError(f"{l[0]} n'a pas de transition.")


ANTLR runtime and generated code versions disagree: 4.13.1!=4.12.0
ANTLR runtime and generated code versions disagree: 4.13.1!=4.12.0
Liste des états : ['S0', 'S1', 'S2']
Liste des gains : ['5', '1', '0']

[Action(a), Action(b), Action(c)]

Transition(State(S0), Action(), [State(S1), State(S2)], [5, 5])
Transition(State(S1), Action(b), [State(S1), State(S0)], [2, 8])
Transition(State(S1), Action(a), [State(S2), State(S0), State(S1)], [1, 3, 6])
Transition(State(S2), Action(c), [State(S0), State(S1)], [5, 5])



# Visualisation

In [13]:
visu_random_walk(states,actions,transitions) 

current_state : State(S0)
next_transitions : [Transition(State(S0), Action(), [State(S1), State(S2)], [5, 5])]
next_action : Action()
next_state : State(S2)
Chemin 1: départ de State(S0) et arrivée à State(S2)

current_state : State(S2)
next_transitions : [Transition(State(S2), Action(), [State(S5), State(S6)], [5, 5])]
next_action : Action()
next_state : State(S5)
Chemin 2: départ de State(S2) et arrivée à State(S5)

current_state : State(S5)
next_transitions : [Transition(State(S5), Action(), [State(D4), State(D5)], [5, 5])]
next_action : Action()
next_state : State(D4)
Chemin 3: départ de State(S5) et arrivée à State(D4)



# smc quantitatif 

In [29]:
eps = 0.01
delt = 0.1
n = 5
S = 'S0'

smc_quantitatif(S,n,eps,delt,states,actions,transitions)

0.32659794356031485


# smc qualitatif

In [30]:
smc_qualitatif(0.1,0.1,0.12,5,'S0',states,actions,transitions)

-0.9542425094393249 0.9542425094393249
plus grand que 0.12


0

# q_learning

In [31]:
q_learning(states,actions,transitions,100,1/2)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
mon état initial est 0
0 [0]
0
[[1. 1. 2.]
 [1. 1. 1.]
 [1. 1. 1.]]
------------
[[0.  0.  2.5]
 [0.  0.  0. ]
 [0.  0.  0. ]]

2 [0, 2]
2
[[1. 1. 2.]
 [1. 1. 1.]
 [1. 2. 1.]]
------------
[[0.    0.    2.5  ]
 [0.    0.    0.   ]
 [0.    0.625 0.   ]]

0 [0, 2, 0]
0
[[1. 2. 2.]
 [1. 1. 1.]
 [1. 2. 1.]]
------------
[[0.      2.65625 2.5    ]
 [0.      0.      0.     ]
 [0.      0.625   0.     ]]

2 [2, 0, 2]
2
[[1. 2. 2.]
 [1. 1. 1.]
 [2. 2. 1.]]
------------
[[0.        2.65625   2.5      ]
 [0.        0.        0.       ]
 [0.6640625 0.625     0.       ]]

0 [0, 2, 0]
0
[[1. 3. 2.]
 [1. 1. 1.]
 [2. 2. 1.]]
------------
[[0.        3.4375    2.5      ]
 [0.        0.        0.       ]
 [0.6640625 0.625     0.       ]]

1 [2, 0, 1]
1
[[1. 3. 2.]
 [1. 1. 2.]
 [2. 2. 1.]]
------------
[[0.        3.4375    2.5      ]
 [0.        0.        1.359375 ]
 [0.6640625 0.625     0.       ]]

0 [0, 1, 0]
0
[[2. 3. 2.]
 [1. 1. 2.]
 [2. 2. 1.]]
------------
[[2

array([[5.30284614, 5.14073933, 5.24140336],
       [2.28313206, 2.59726531, 2.34934777],
       [1.37015695, 1.56275934, 1.44625647]])

# accessibilité MC

In [32]:
state = input ("quel état ?")
accessibilite(state,transitions, states)

S0 = [] ; S1 = ['S0'] ; S? = ['S1', 'S2']


# accessibilite MDP

In [33]:
#state = input ("quel état ?")
#adversaire = input("quel adversaire")

visu_random_walk(states,actions,suppr(transitions,['a']*16,statesid)) 
accessibilite_mdp('S11',transitions,states,['a']*16)

current_state : State(S0)
next_transitions : []
Chemin 1: départ de State(S0) et arrivée à State(S0)

current_state : State(S0)
next_transitions : []
Chemin 2: départ de State(S0) et arrivée à State(S0)

current_state : State(S0)
next_transitions : []
Chemin 3: départ de State(S0) et arrivée à State(S0)

['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
[Transition(State(S1), Action(a), [State(S2), State(S0), State(S1)], [1, 3, 6])]
S0 = ['S0', 'S1', 'S2'] ; S1 = ['S11'] ; S? = []


# itération de valeurs

In [34]:
iteration_valeur(0.1,0.5,states,actions,transitions)

[5. 1. 0.] ['', Action(b), Action(c)]


(['', Action(b), Action(c)], array([5., 1., 0.]))